In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 5       # number of variables
num_ineq = 5      # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msLinear
model = msLinear(num_var, num_ineq, timelimit=60)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model.solve("gurobi")
    tock = time.time()
    # eval
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_exact_5-5.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.60it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean    -0.070980               0.0      0.130315
std      0.623915               0.0      0.012192
min     -1.327442               0.0      0.117002
25%     -0.547070               0.0      0.122635
50%     -0.048624               0.0      0.124088
75%      0.279931               0.0      0.138188
max      1.536205               0.0      0.193369
Number of infeasible solution: 0


## Heuristic - Round

In [8]:
from src.heuristic import naive_round

In [9]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("gurobi")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_heur_rnd_5-5.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.63it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    -0.145474          0.100411      0.124833
std      0.628750          0.062869      0.006676
min     -1.393076          0.001241      0.115513
25%     -0.616161          0.051058      0.119211
50%     -0.138401          0.096044      0.121339
75%      0.249987          0.141277      0.131724
max      1.489957          0.248614      0.137358
Number of infeasible solution: 100


## Heuristic - N1

In [10]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model_heur.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("gurobi")
    tock = time.time()
    # eval
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_heur_n1_5-5.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:12<00:00,  7.98it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean     2.532599               0.0      0.124139
std      0.752820               0.0      0.003805
min      0.485147               0.0      0.120161
25%      2.031729               0.0      0.122271
50%      2.562566               0.0      0.123329
75%      3.124931               0.0      0.124402
max      4.055409               0.0      0.141285
Number of infeasible solution: 0


## Learnable Rounding

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 16            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmLinear
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmLinear(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 132.72
Epoch 1, Validation Loss: 13.71
Epoch 2, Validation Loss: 2.32
Epoch 3, Validation Loss: 2.29
Epoch 4, Validation Loss: 2.10
Epoch 5, Validation Loss: 2.03
Epoch 6, Validation Loss: 1.95
Epoch 7, Validation Loss: 1.88
Epoch 8, Validation Loss: 1.85
Epoch 9, Validation Loss: 1.79
Epoch 10, Validation Loss: 1.72
Epoch 11, Validation Loss: 1.73
Epoch 12, Validation Loss: 1.66
Epoch 13, Validation Loss: 1.63
Epoch 14, Validation Loss: 1.78
Epoch 15, Validation Loss: 1.60
Epoch 16, Validation Loss: 1.55
Epoch 17, Validation Loss: 1.54
Epoch 18, Validation Loss: 1.50
Epoch 19, Validation Loss: 1.43
Epoch 20, Validation Loss: 1.47
Epoch 21, Validation Loss: 1.44
Epoch 22, Validation Loss: 1.50
Epoch 23, Validation Loss: 1.42
Epoch 24, Validation Loss: 1.38
Epoch 25, Validation Loss: 1.59
Epoch 26, Validation Loss: 1.56
Epoch 27, Validation Loss: 1.32
Epoch 28, Validation Loss: 1.35
Epoch 29, Validation Loss: 1.40
Epoch 30, Validation Loss: 1.31
Epoch 31, Valid

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_lr_5-5.csv")

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 261.81it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean     0.477344               0.0      0.002728
std      0.586919               0.0      0.000714
min     -0.625008               0.0      0.001528
25%      0.007256               0.0      0.002011
50%      0.444195               0.0      0.002535
75%      0.845979               0.0      0.003121
max      1.849546               0.0      0.005025
Number of infeasible solution: 0


## Learnable Threshold

In [17]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [18]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 16            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [19]:
# set problem
import neuromancer as nm
from src.problem import nmLinear
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmLinear(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [20]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 126.15
Epoch 1, Validation Loss: 18.77
Epoch 2, Validation Loss: 2.42
Epoch 3, Validation Loss: 2.27
Epoch 4, Validation Loss: 2.13
Epoch 5, Validation Loss: 2.03
Epoch 6, Validation Loss: 1.90
Epoch 7, Validation Loss: 2.07
Epoch 8, Validation Loss: 1.84
Epoch 9, Validation Loss: 1.98
Epoch 10, Validation Loss: 1.73
Epoch 11, Validation Loss: 1.74
Epoch 12, Validation Loss: 1.71
Epoch 13, Validation Loss: 1.62
Epoch 14, Validation Loss: 1.59
Epoch 15, Validation Loss: 1.58
Epoch 16, Validation Loss: 1.78
Epoch 17, Validation Loss: 1.50
Epoch 18, Validation Loss: 1.47
Epoch 19, Validation Loss: 1.47
Epoch 20, Validation Loss: 1.46
Epoch 21, Validation Loss: 1.70
Epoch 22, Validation Loss: 1.45
Epoch 23, Validation Loss: 1.39
Epoch 24, Validation Loss: 1.37
Epoch 25, Validation Loss: 1.35
Epoch 26, Validation Loss: 1.29
Epoch 27, Validation Loss: 1.27
Epoch 28, Validation Loss: 1.27
Epoch 29, Validation Loss: 1.50
Epoch 30, Validation Loss: 1.30
Epoch 31, Valid

In [21]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_lt_5-5.csv")

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 248.29it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean     0.369391          0.000884      0.002948
std      0.616343          0.007562      0.000754
min     -0.816265          0.000000      0.001503
25%     -0.069515          0.000000      0.002145
50%      0.354425          0.000000      0.003003
75%      0.775363          0.000000      0.003295
max      1.773446          0.075239      0.005105
Number of infeasible solution: 4


### Parametric Learning Then Rounding

In [22]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [23]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 16            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [24]:
# set problem
import neuromancer as nm
from src.problem import nmLinear
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmLinear(["b", "x"], num_var, num_ineq, penalty_weight)

In [25]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 126.07
Epoch 1, Validation Loss: 12.38
Epoch 2, Validation Loss: 2.50
Epoch 3, Validation Loss: 2.37
Epoch 4, Validation Loss: 2.21
Epoch 5, Validation Loss: 2.12
Epoch 6, Validation Loss: 1.98
Epoch 7, Validation Loss: 2.01
Epoch 8, Validation Loss: 1.91
Epoch 9, Validation Loss: 1.83
Epoch 10, Validation Loss: 1.76
Epoch 11, Validation Loss: 1.72
Epoch 12, Validation Loss: 1.69
Epoch 13, Validation Loss: 1.61
Epoch 14, Validation Loss: 1.59
Epoch 15, Validation Loss: 1.56
Epoch 16, Validation Loss: 1.53
Epoch 17, Validation Loss: 1.48
Epoch 18, Validation Loss: 1.45
Epoch 19, Validation Loss: 1.44
Epoch 20, Validation Loss: 1.41
Epoch 21, Validation Loss: 1.34
Epoch 22, Validation Loss: 1.28
Epoch 23, Validation Loss: 1.25
Epoch 24, Validation Loss: 1.14
Epoch 25, Validation Loss: 1.11
Epoch 26, Validation Loss: 1.05
Epoch 27, Validation Loss: 0.98
Epoch 28, Validation Loss: 1.01
Epoch 29, Validation Loss: 0.91
Epoch 30, Validation Loss: 0.92
Epoch 31, Valid

In [26]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_pr_5-5.csv")

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 457.61it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean     0.157925          0.019531      0.000661
std      0.630261          0.026569      0.000545
min     -1.032693          0.000000      0.000000
25%     -0.342312          0.000000      0.000000
50%      0.134601          0.005831      0.001000
75%      0.615587          0.032797      0.001010
max      1.594563          0.128844      0.002000
Number of infeasible solution: 56
